# ===============================================================================
# ЭКСПЕРИМЕНТЫ: CATBOOST & LIGHTGBM С МЕТОДАМИ БАЛАНСИРОВКИ
# ===============================================================================

**Цель:** Протестировать все комбинации алгоритмов и методов балансировки

**Эксперименты:**
- **Алгоритмы:** CatBoost, LightGBM
- **Методы балансировки:** No balancing, Class weights, SMOTE, Undersampling, SMOTE+Undersampling
- **Сегменты:** Segment 1, Segment 2
- **Всего экспериментов:** 2 × 5 × 2 = **20**

**Метрики:** ROC-AUC, Gini, PR-AUC, Precision, Recall, F1, Confusion Matrix

**Дата:** 2025-01-13  
**Random seed:** 42

# ===============================================================================

---
# 1. ИМПОРТ БИБЛИОТЕК И КОНФИГУРАЦИЯ

In [1]:
# ====================================================================================
# ИМПОРТ БИБЛИОТЕК
# ====================================================================================

import os
import warnings
from datetime import datetime
from pathlib import Path
import time
import pickle
import gc

# Данные
import numpy as np
import pandas as pd

# Progress bars
from tqdm.auto import tqdm

# ML Models
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier

# Balancing
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_class_weight

# Metrics
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve,
    classification_report, confusion_matrix,
    accuracy_score, f1_score, precision_score, recall_score
)

# Настройки
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("="*80)
print("ЭКСПЕРИМЕНТЫ: CATBOOST & LIGHTGBM")
print("="*80)
print(f"✓ Библиотеки импортированы")
print(f"  Дата запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

ЭКСПЕРИМЕНТЫ: CATBOOST & LIGHTGBM
✓ Библиотеки импортированы
  Дата запуска: 2025-11-16 21:02:01


In [2]:
# ====================================================================================
# КОНФИГУРАЦИЯ
# ====================================================================================

class Config:
    """Централизованная конфигурация для экспериментов"""
    
    # ВОСПРОИЗВОДИМОСТЬ
    RANDOM_SEED = 42
    
    # ПУТИ
    OUTPUT_DIR = Path("output")
    MODELS_DIR = Path("models")
    
    # КОЛОНКИ
    TARGET_COLUMN = 'target_churn_3m'
    
    # СЕГМЕНТЫ
    SEGMENTS = {
        'Segment 1': {
            'name': 'Small Business',
            'train': 'seg1_train.parquet',
            'val': 'seg1_val.parquet',
            'test': 'seg1_test.parquet'
        },
        'Segment 2': {
            'name': 'Middle + Large Business',
            'train': 'seg2_train.parquet',
            'val': 'seg2_val.parquet',
            'test': 'seg2_test.parquet'
        }
    }
    
    # АЛГОРИТМЫ
    ALGORITHMS = ['CatBoost', 'LightGBM']
    
    # МЕТОДЫ БАЛАНСИРОВКИ
    BALANCING_METHODS = [
        'No balancing',
        'Class weights',
        'SMOTE',
        'Random Undersampling',
        'SMOTE + Undersampling'
    ]
    
    # CATBOOST ПАРАМЕТРЫ
    CATBOOST_PARAMS = {
        'iterations': 300,
        'depth': 6,
        'learning_rate': 0.05,
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'early_stopping_rounds': 50,
        'use_best_model': True,
        'random_seed': 42,
        'task_type': 'CPU',
        'verbose': False,
        'allow_writing_files': False
    }
    
    # LIGHTGBM ПАРАМЕТРЫ
    LIGHTGBM_PARAMS = {
        'n_estimators': 300,
        'max_depth': 6,
        'learning_rate': 0.05,
        'objective': 'binary',
        'metric': 'auc',
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1
    }
    
    # BALANCING ПАРАМЕТРЫ
    SMOTE_PARAMS = {
        'random_state': 42,
        'k_neighbors': 5
    }
    
    UNDERSAMPLING_PARAMS = {
        'random_state': 42,
        'sampling_strategy': 'auto'
    }
    
    # THRESHOLD TUNING
    THRESHOLD_METRIC = 'f1'
    
    @classmethod
    def create_directories(cls):
        for dir_path in [cls.OUTPUT_DIR, cls.MODELS_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)

config = Config()
config.create_directories()
np.random.seed(config.RANDOM_SEED)

print("\n✓ Конфигурация инициализирована")
print(f"  Random seed: {config.RANDOM_SEED}")
print(f"  Алгоритмы: {', '.join(config.ALGORITHMS)}")
print(f"  Методы балансировки: {len(config.BALANCING_METHODS)}")
print(f"  Сегментов: {len(config.SEGMENTS)}")
print(f"  Всего экспериментов: {len(config.ALGORITHMS) * len(config.BALANCING_METHODS) * len(config.SEGMENTS)}")


✓ Конфигурация инициализирована
  Random seed: 42
  Алгоритмы: CatBoost, LightGBM
  Методы балансировки: 5
  Сегментов: 2
  Всего экспериментов: 20


---
# 2. ЗАГРУЗКА ДАННЫХ

In [3]:
# ====================================================================================
# ЗАГРУЗКА ДАННЫХ
# ====================================================================================

print("\n" + "="*80)
print("ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ")
print("="*80)

data = {}

for seg_id, seg_info in config.SEGMENTS.items():
    print(f"\n{seg_id}: {seg_info['name']}")
    print("-" * 80)
    
    data[seg_id] = {}
    
    for split in ['train', 'val', 'test']:
        file_path = config.OUTPUT_DIR / seg_info[split]
        
        if not file_path.exists():
            raise FileNotFoundError(
                f"Файл не найден: {file_path}\n"
                f"Сначала запустите notebook 01_data_preparation_eda.ipynb"
            )
        
        df = pd.read_parquet(file_path)
        data[seg_id][split] = df
        
        churn_rate = df[config.TARGET_COLUMN].mean()
        print(f"  {split.upper():5s}: {df.shape} | Churn: {churn_rate*100:.2f}%")

print("\n" + "="*80)
print("✓ Все данные загружены успешно")
print("="*80)


ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ

Segment 1: Small Business
--------------------------------------------------------------------------------
  TRAIN: (2024010, 121) | Churn: 1.51%
  VAL  : (493087, 121) | Churn: 1.86%
  TEST : (500370, 121) | Churn: 1.50%

Segment 2: Middle + Large Business
--------------------------------------------------------------------------------
  TRAIN: (129397, 122) | Churn: 0.50%
  VAL  : (38793, 122) | Churn: 0.71%
  TEST : (42532, 122) | Churn: 0.78%

✓ Все данные загружены успешно


---
# 3. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ

In [4]:
# ====================================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ====================================================================================

def prepare_data(df, target_col):
    """Разделение на X и y"""
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return X, y


def apply_balancing(X_train, y_train, method):
    """
    Применить метод балансировки к train данным.
    
    Returns:
    --------
    X_balanced, y_balanced, sample_weights (or None)
    """
    
    if method == 'No balancing':
        return X_train.copy(), y_train.copy(), None
    
    elif method == 'Class weights':
        # Вычисляем веса для каждого сэмпла
        class_weights = compute_class_weight(
            'balanced',
            classes=np.unique(y_train),
            y=y_train
        )
        sample_weights = np.array([class_weights[int(y)] for y in y_train])
        return X_train.copy(), y_train.copy(), sample_weights
    
    elif method == 'SMOTE':
        smote = SMOTE(**config.SMOTE_PARAMS)
        X_res, y_res = smote.fit_resample(X_train, y_train)
        return X_res, y_res, None
    
    elif method == 'Random Undersampling':
        rus = RandomUnderSampler(**config.UNDERSAMPLING_PARAMS)
        X_res, y_res = rus.fit_resample(X_train, y_train)
        return X_res, y_res, None
    
    elif method == 'SMOTE + Undersampling':
        # Сначала SMOTE, потом undersampling
        smote = SMOTE(**config.SMOTE_PARAMS)
        X_smote, y_smote = smote.fit_resample(X_train, y_train)
        
        rus = RandomUnderSampler(**config.UNDERSAMPLING_PARAMS)
        X_res, y_res = rus.fit_resample(X_smote, y_smote)
        
        return X_res, y_res, None
    
    else:
        raise ValueError(f"Unknown balancing method: {method}")


def train_model(algorithm, X_train, y_train, X_val, y_val, sample_weights=None):
    """
    Обучить модель.
    
    Returns:
    --------
    trained_model
    """
    
    if algorithm == 'CatBoost':
        model = CatBoostClassifier(**config.CATBOOST_PARAMS)
        
        train_pool = Pool(X_train, y_train, weight=sample_weights)
        val_pool = Pool(X_val, y_val)
        
        model.fit(train_pool, eval_set=val_pool)
        
    elif algorithm == 'LightGBM':
        model = LGBMClassifier(**config.LIGHTGBM_PARAMS)
        
        model.fit(
            X_train, y_train,
            sample_weight=sample_weights,
            eval_set=[(X_val, y_val)],
            callbacks=[]
        )
    
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")
    
    return model


def find_optimal_threshold(y_true, y_pred_proba, metric='f1'):
    """
    Найти оптимальный порог классификации.
    
    Returns:
    --------
    optimal_threshold, best_score
    """
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        
        if metric == 'f1':
            score = f1_score(y_true, y_pred, zero_division=0)
        elif metric == 'recall':
            score = recall_score(y_true, y_pred, zero_division=0)
        elif metric == 'precision':
            score = precision_score(y_true, y_pred, zero_division=0)
        else:
            raise ValueError(f"Unknown metric: {metric}")
        
        scores.append(score)
    
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx], scores[optimal_idx]


def calculate_metrics(y_true, y_pred_proba, y_pred, threshold):
    """
    Рассчитать все метрики.
    
    Returns:
    --------
    dict with metrics
    """
    metrics = {
        'threshold': threshold,
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'pr_auc': average_precision_score(y_true, y_pred_proba),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
    }
    
    metrics['gini'] = 2 * metrics['roc_auc'] - 1
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    metrics['tn'] = cm[0, 0]
    metrics['fp'] = cm[0, 1]
    metrics['fn'] = cm[1, 0]
    metrics['tp'] = cm[1, 1]
    
    return metrics


print("✓ Вспомогательные функции определены")

✓ Вспомогательные функции определены


---
# 4. ЗАПУСК ЭКСПЕРИМЕНТОВ

Тестируем все комбинации:  
**2 алгоритма × 5 методов балансировки × 2 сегмента = 20 экспериментов**

In [5]:
# ====================================================================================
# ЗАПУСК ВСЕХ ЭКСПЕРИМЕНТОВ
# ====================================================================================

print("\n" + "="*80)
print("ЗАПУСК ЭКСПЕРИМЕНТОВ")
print("="*80)

results = []
models_storage = {}  # Для хранения лучших моделей

# Общее количество экспериментов
total_experiments = len(config.ALGORITHMS) * len(config.BALANCING_METHODS) * len(config.SEGMENTS)

print(f"\nВсего экспериментов: {total_experiments}")
print(f"Начало: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*80)

# Создаем progress bar
pbar = tqdm(total=total_experiments, desc="Общий прогресс", position=0)

experiment_num = 0

# Цикл по сегментам
for seg_id, seg_info in config.SEGMENTS.items():
    
    # Подготовка данных для сегмента
    X_train, y_train = prepare_data(data[seg_id]['train'], config.TARGET_COLUMN)
    X_val, y_val = prepare_data(data[seg_id]['val'], config.TARGET_COLUMN)
    X_test, y_test = prepare_data(data[seg_id]['test'], config.TARGET_COLUMN)
    
    # Инициализируем хранилище для моделей этого сегмента
    models_storage[seg_id] = {}
    
    # Цикл по алгоритмам
    for algorithm in config.ALGORITHMS:
        
        models_storage[seg_id][algorithm] = {
            'best_model': None,
            'best_roc_auc': 0,
            'best_method': None
        }
        
        # Цикл по методам балансировки
        for balancing_method in config.BALANCING_METHODS:
            
            experiment_num += 1
            
            try:
                # ===== 1. БАЛАНСИРОВКА =====
                X_train_balanced, y_train_balanced, sample_weights = apply_balancing(
                    X_train, y_train, balancing_method
                )
                
                # ===== 2. ОБУЧЕНИЕ =====
                start_time = time.time()
                
                model = train_model(
                    algorithm,
                    X_train_balanced, y_train_balanced,
                    X_val, y_val,
                    sample_weights
                )
                
                train_time = time.time() - start_time
                
                # ===== 3. ПРЕДСКАЗАНИЯ =====
                y_val_proba = model.predict_proba(X_val)[:, 1]
                y_test_proba = model.predict_proba(X_test)[:, 1]
                
                # ===== 4. OPTIMAL THRESHOLD =====
                optimal_threshold, _ = find_optimal_threshold(
                    y_val, y_val_proba, config.THRESHOLD_METRIC
                )
                
                # Предсказания с optimal threshold на test
                y_test_pred = (y_test_proba >= optimal_threshold).astype(int)
                
                # ===== 5. МЕТРИКИ НА TEST =====
                metrics = calculate_metrics(
                    y_test, y_test_proba, y_test_pred, optimal_threshold
                )
                
                # ===== 6. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ =====
                result = {
                    'segment_group': seg_id,
                    'segment_name': seg_info['name'],
                    'algorithm': algorithm,
                    'balancing_method': balancing_method,
                    'train_samples': len(X_train_balanced),
                    'train_time_sec': train_time,
                    **metrics
                }
                
                results.append(result)
                
                # ===== 7. ОБНОВЛЕНИЕ ЛУЧШЕЙ МОДЕЛИ =====
                if metrics['roc_auc'] > models_storage[seg_id][algorithm]['best_roc_auc']:
                    models_storage[seg_id][algorithm]['best_model'] = model
                    models_storage[seg_id][algorithm]['best_roc_auc'] = metrics['roc_auc']
                    models_storage[seg_id][algorithm]['best_method'] = balancing_method
                
                # ===== 8. ВЫВОД ПРОГРЕССА =====
                pbar.set_postfix_str(
                    f"{seg_id} | {algorithm} | {balancing_method[:15]:15s} | "
                    f"ROC-AUC: {metrics['roc_auc']:.4f} | Time: {train_time:.1f}s"
                )
                pbar.update(1)
                
                # Очистка памяти
                del X_train_balanced, y_train_balanced, sample_weights
                if balancing_method != 'Class weights' and balancing_method != 'No balancing':
                    gc.collect()
                
            except Exception as e:
                print(f"\n❌ Ошибка в эксперименте {experiment_num}: {seg_id} | {algorithm} | {balancing_method}")
                print(f"   Ошибка: {str(e)}")
                pbar.update(1)
                continue

pbar.close()

print("\n" + "="*80)
print(f"✓ Все эксперименты завершены")
print(f"  Время окончания: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"  Успешных экспериментов: {len(results)}/{total_experiments}")
print("="*80)


ЗАПУСК ЭКСПЕРИМЕНТОВ

Всего экспериментов: 20
Начало: 2025-11-16 21:02:02



Общий прогресс:   0%|          | 0/20 [00:00<?, ?it/s]


✓ Все эксперименты завершены
  Время окончания: 2025-11-16 21:12:31
  Успешных экспериментов: 20/20


---
# 5. АНАЛИЗ РЕЗУЛЬТАТОВ

In [6]:
# ====================================================================================
# СОЗДАНИЕ ТАБЛИЦЫ РЕЗУЛЬТАТОВ
# ====================================================================================

print("\n" + "="*80)
print("АНАЛИЗ РЕЗУЛЬТАТОВ")
print("="*80)

# Создаем DataFrame с результатами
results_df = pd.DataFrame(results)

# Сортируем по ROC-AUC
results_df = results_df.sort_values('roc_auc', ascending=False).reset_index(drop=True)

print(f"\nВсего результатов: {len(results_df)}")
print(f"\nСтолбцы: {list(results_df.columns)}")

# Основные статистики
print(f"\nОСНОВНЫЕ СТАТИСТИКИ:")
print(f"  ROC-AUC: min={results_df['roc_auc'].min():.4f}, "
      f"max={results_df['roc_auc'].max():.4f}, "
      f"mean={results_df['roc_auc'].mean():.4f}")
print(f"  Gini: min={results_df['gini'].min():.4f}, "
      f"max={results_df['gini'].max():.4f}, "
      f"mean={results_df['gini'].mean():.4f}")
print(f"  F1: min={results_df['f1'].min():.4f}, "
      f"max={results_df['f1'].max():.4f}, "
      f"mean={results_df['f1'].mean():.4f}")

print("\n" + "="*80)


АНАЛИЗ РЕЗУЛЬТАТОВ

Всего результатов: 20

Столбцы: ['segment_group', 'segment_name', 'algorithm', 'balancing_method', 'train_samples', 'train_time_sec', 'threshold', 'roc_auc', 'pr_auc', 'precision', 'recall', 'f1', 'gini', 'tn', 'fp', 'fn', 'tp']

ОСНОВНЫЕ СТАТИСТИКИ:
  ROC-AUC: min=0.8447, max=0.8954, mean=0.8716
  Gini: min=0.6894, max=0.7908, mean=0.7431
  F1: min=0.0563, max=0.2501, mean=0.1669



In [7]:
# ====================================================================================
# ТОП-5 РЕЗУЛЬТАТОВ ПО СЕГМЕНТАМ
# ====================================================================================

print("\n" + "="*80)
print("ТОП-5 РЕЗУЛЬТАТОВ ПО ROC-AUC (для каждого сегмента)")
print("="*80)

for seg_id in config.SEGMENTS.keys():
    seg_results = results_df[results_df['segment_group'] == seg_id].head(5)
    
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    
    display_cols = ['algorithm', 'balancing_method', 'roc_auc', 'gini', 'f1', 
                   'precision', 'recall', 'train_time_sec']
    
    print(seg_results[display_cols].to_string(index=False))

print("\n" + "="*80)


ТОП-5 РЕЗУЛЬТАТОВ ПО ROC-AUC (для каждого сегмента)

Segment 1: Small Business
--------------------------------------------------------------------------------
algorithm     balancing_method  roc_auc   gini     f1  precision  recall  train_time_sec
 LightGBM        Class weights   0.8954 0.7908 0.2501     0.2397  0.2615         15.4536
 LightGBM         No balancing   0.8950 0.7900 0.2491     0.2495  0.2487         14.5066
 LightGBM Random Undersampling   0.8925 0.7850 0.2472     0.2376  0.2575          7.7819
 CatBoost        Class weights   0.8924 0.7848 0.2464     0.2542  0.2390         40.1730
 CatBoost Random Undersampling   0.8909 0.7817 0.2424     0.2337  0.2518         12.7865

Segment 2: Middle + Large Business
--------------------------------------------------------------------------------
algorithm     balancing_method  roc_auc   gini     f1  precision  recall  train_time_sec
 CatBoost         No balancing   0.8769 0.7537 0.0684     0.1030  0.0512          4.7972
 CatBoost 

In [8]:
# ====================================================================================
# ЛУЧШИЕ РЕЗУЛЬТАТЫ ПО АЛГОРИТМАМ
# ====================================================================================

print("\n" + "="*80)
print("ЛУЧШИЕ РЕЗУЛЬТАТЫ ПО АЛГОРИТМАМ")
print("="*80)

for algorithm in config.ALGORITHMS:
    print(f"\n{algorithm}:")
    print("-" * 80)
    
    algo_results = results_df[results_df['algorithm'] == algorithm]
    
    for seg_id in config.SEGMENTS.keys():
        seg_algo = algo_results[algo_results['segment_group'] == seg_id]
        
        if len(seg_algo) > 0:
            best = seg_algo.iloc[0]
            print(f"  {seg_id}: {best['balancing_method']:25s} | "
                  f"ROC-AUC: {best['roc_auc']:.4f} | "
                  f"Gini: {best['gini']:.4f} | "
                  f"F1: {best['f1']:.4f}")

print("\n" + "="*80)


ЛУЧШИЕ РЕЗУЛЬТАТЫ ПО АЛГОРИТМАМ

CatBoost:
--------------------------------------------------------------------------------
  Segment 1: Class weights             | ROC-AUC: 0.8924 | Gini: 0.7848 | F1: 0.2464
  Segment 2: No balancing              | ROC-AUC: 0.8769 | Gini: 0.7537 | F1: 0.0684

LightGBM:
--------------------------------------------------------------------------------
  Segment 1: Class weights             | ROC-AUC: 0.8954 | Gini: 0.7908 | F1: 0.2501
  Segment 2: No balancing              | ROC-AUC: 0.8649 | Gini: 0.7299 | F1: 0.0563



In [9]:
# ====================================================================================
# СРЕДНИЕ РЕЗУЛЬТАТЫ ПО МЕТОДАМ БАЛАНСИРОВКИ
# ====================================================================================

print("\n" + "="*80)
print("СРЕДНИЕ РЕЗУЛЬТАТЫ ПО МЕТОДАМ БАЛАНСИРОВКИ")
print("="*80)

balancing_stats = results_df.groupby('balancing_method').agg({
    'roc_auc': ['mean', 'std', 'min', 'max'],
    'gini': ['mean', 'std'],
    'f1': ['mean', 'std'],
    'train_time_sec': ['mean', 'std']
}).round(4)

balancing_stats = balancing_stats.sort_values(('roc_auc', 'mean'), ascending=False)

print("\nСредние метрики по методам балансировки:")
print(balancing_stats)

print("\n" + "="*80)


СРЕДНИЕ РЕЗУЛЬТАТЫ ПО МЕТОДАМ БАЛАНСИРОВКИ

Средние метрики по методам балансировки:
                      roc_auc                        gini            f1  \
                         mean    std    min    max   mean    std   mean   
balancing_method                                                          
No balancing           0.8815 0.0134 0.8649 0.8950 0.7630 0.0267 0.1549   
Random Undersampling   0.8766 0.0180 0.8561 0.8925 0.7531 0.0360 0.1663   
Class weights          0.8758 0.0226 0.8473 0.8954 0.7516 0.0452 0.1782   
SMOTE                  0.8622 0.0152 0.8488 0.8762 0.7244 0.0303 0.1683   
SMOTE + Undersampling  0.8618 0.0159 0.8447 0.8766 0.7236 0.0318 0.1668   

                             train_time_sec          
                         std           mean     std  
balancing_method                                     
No balancing          0.1070        15.0910 16.9822  
Random Undersampling  0.0911         5.5717  5.8139  
Class weights         0.0810        14.8844

---
# 6. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ И МОДЕЛЕЙ

In [10]:
# ====================================================================================
# СОХРАНЕНИЕ РЕЗУЛЬТАТОВ В CSV
# ====================================================================================

print("\n" + "="*80)
print("СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
print("="*80)

# Сохраняем таблицу результатов
results_file = config.OUTPUT_DIR / 'experiments_part1.csv'
results_df.to_csv(results_file, index=False)

print(f"\n✓ Результаты сохранены: {results_file}")
print(f"  Всего экспериментов: {len(results_df)}")
print(f"  Размер файла: {results_file.stat().st_size / 1024:.2f} KB")


СОХРАНЕНИЕ РЕЗУЛЬТАТОВ

✓ Результаты сохранены: output\experiments_part1.csv
  Всего экспериментов: 20
  Размер файла: 4.69 KB


In [11]:
# ====================================================================================
# СОХРАНЕНИЕ ЛУЧШИХ МОДЕЛЕЙ
# ====================================================================================

print("\n" + "="*80)
print("СОХРАНЕНИЕ ЛУЧШИХ МОДЕЛЕЙ")
print("="*80)

saved_models = []

for seg_id in config.SEGMENTS.keys():
    seg_num = seg_id.split()[1]  # '1' or '2'
    
    for algorithm in config.ALGORITHMS:
        
        model_info = models_storage[seg_id][algorithm]
        
        if model_info['best_model'] is not None:
            
            # Формируем имя файла
            algo_name = algorithm.lower().replace(' ', '_')
            model_filename = f"best_{algo_name}_seg{seg_num}.pkl"
            model_path = config.MODELS_DIR / model_filename
            
            # Сохраняем модель
            with open(model_path, 'wb') as f:
                pickle.dump(model_info['best_model'], f)
            
            saved_models.append({
                'file': model_filename,
                'segment': seg_id,
                'algorithm': algorithm,
                'balancing_method': model_info['best_method'],
                'roc_auc': model_info['best_roc_auc']
            })
            
            print(f"\n✓ {model_filename}")
            print(f"  Сегмент: {seg_id}")
            print(f"  Алгоритм: {algorithm}")
            print(f"  Метод балансировки: {model_info['best_method']}")
            print(f"  ROC-AUC (test): {model_info['best_roc_auc']:.4f}")
            print(f"  Размер файла: {model_path.stat().st_size / 1024:.2f} KB")

print("\n" + "="*80)
print(f"✓ Всего сохранено моделей: {len(saved_models)}")
print("="*80)


СОХРАНЕНИЕ ЛУЧШИХ МОДЕЛЕЙ

✓ best_catboost_seg1.pkl
  Сегмент: Segment 1
  Алгоритм: CatBoost
  Метод балансировки: Class weights
  ROC-AUC (test): 0.8924
  Размер файла: 4696.85 KB

✓ best_lightgbm_seg1.pkl
  Сегмент: Segment 1
  Алгоритм: LightGBM
  Метод балансировки: Class weights
  ROC-AUC (test): 0.8954
  Размер файла: 1058.56 KB

✓ best_catboost_seg2.pkl
  Сегмент: Segment 2
  Алгоритм: CatBoost
  Метод балансировки: No balancing
  ROC-AUC (test): 0.8769
  Размер файла: 611.54 KB

✓ best_lightgbm_seg2.pkl
  Сегмент: Segment 2
  Алгоритм: LightGBM
  Метод балансировки: No balancing
  ROC-AUC (test): 0.8649
  Размер файла: 1024.30 KB

✓ Всего сохранено моделей: 4


In [12]:
# ====================================================================================
# СОХРАНЕНИЕ МЕТАДАННЫХ
# ====================================================================================

import json

metadata = {
    'created_date': datetime.now().isoformat(),
    'random_seed': config.RANDOM_SEED,
    'total_experiments': len(results_df),
    'algorithms': config.ALGORITHMS,
    'balancing_methods': config.BALANCING_METHODS,
    'segments': list(config.SEGMENTS.keys()),
    'catboost_params': config.CATBOOST_PARAMS,
    'lightgbm_params': config.LIGHTGBM_PARAMS,
    'saved_models': saved_models,
    'best_overall': {
        'segment': results_df.iloc[0]['segment_group'],
        'algorithm': results_df.iloc[0]['algorithm'],
        'balancing_method': results_df.iloc[0]['balancing_method'],
        'roc_auc': float(results_df.iloc[0]['roc_auc']),
        'gini': float(results_df.iloc[0]['gini']),
        'f1': float(results_df.iloc[0]['f1'])
    }
}

metadata_file = config.OUTPUT_DIR / 'experiments_part1_metadata.json'
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\n✓ Метаданные сохранены: {metadata_file}")


✓ Метаданные сохранены: output\experiments_part1_metadata.json


---
# 7. ФИНАЛЬНАЯ СВОДКА

In [13]:
# ====================================================================================
# ФИНАЛЬНАЯ СВОДКА
# ====================================================================================

print("\n\n" + "="*80)
print("✓✓✓ ЭКСПЕРИМЕНТЫ ЗАВЕРШЕНЫ УСПЕШНО ✓✓✓")
print("="*80)

print(f"\nДата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Random seed: {config.RANDOM_SEED}")

print(f"\n{'='*80}")
print("СТАТИСТИКА ЭКСПЕРИМЕНТОВ")
print(f"{'='*80}")

print(f"\nВсего экспериментов: {len(results_df)}")
print(f"Алгоритмов: {len(config.ALGORITHMS)}")
print(f"Методов балансировки: {len(config.BALANCING_METHODS)}")
print(f"Сегментов: {len(config.SEGMENTS)}")

print(f"\n{'='*80}")
print("ЛУЧШИЕ РЕЗУЛЬТАТЫ")
print(f"{'='*80}")

best_overall = results_df.iloc[0]
print(f"\nЛучший результат (по ROC-AUC):")
print(f"  Сегмент: {best_overall['segment_group']} ({best_overall['segment_name']})")
print(f"  Алгоритм: {best_overall['algorithm']}")
print(f"  Метод балансировки: {best_overall['balancing_method']}")
print(f"  ROC-AUC: {best_overall['roc_auc']:.4f}")
print(f"  Gini: {best_overall['gini']:.4f}")
print(f"  F1-Score: {best_overall['f1']:.4f}")
print(f"  Precision: {best_overall['precision']:.4f}")
print(f"  Recall: {best_overall['recall']:.4f}")

print(f"\n{'='*80}")
print("СОХРАНЕННЫЕ ФАЙЛЫ")
print(f"{'='*80}")

print(f"\nРЕЗУЛЬТАТЫ (output/):")
print(f"  • experiments_part1.csv - таблица всех {len(results_df)} экспериментов")
print(f"  • experiments_part1_metadata.json - метаданные")

print(f"\nМОДЕЛИ (models/):")
for model in saved_models:
    print(f"  • {model['file']} - {model['segment']} | {model['algorithm']} | "
          f"{model['balancing_method']} | ROC-AUC: {model['roc_auc']:.4f}")

print(f"\n{'='*80}")
print("СЛЕДУЮЩИЙ ШАГ: Сравнительный анализ и визуализация результатов")
print(f"{'='*80}")



✓✓✓ ЭКСПЕРИМЕНТЫ ЗАВЕРШЕНЫ УСПЕШНО ✓✓✓

Дата: 2025-11-16 21:12:32
Random seed: 42

СТАТИСТИКА ЭКСПЕРИМЕНТОВ

Всего экспериментов: 20
Алгоритмов: 2
Методов балансировки: 5
Сегментов: 2

ЛУЧШИЕ РЕЗУЛЬТАТЫ

Лучший результат (по ROC-AUC):
  Сегмент: Segment 1 (Small Business)
  Алгоритм: LightGBM
  Метод балансировки: Class weights
  ROC-AUC: 0.8954
  Gini: 0.7908
  F1-Score: 0.2501
  Precision: 0.2397
  Recall: 0.2615

СОХРАНЕННЫЕ ФАЙЛЫ

РЕЗУЛЬТАТЫ (output/):
  • experiments_part1.csv - таблица всех 20 экспериментов
  • experiments_part1_metadata.json - метаданные

МОДЕЛИ (models/):
  • best_catboost_seg1.pkl - Segment 1 | CatBoost | Class weights | ROC-AUC: 0.8924
  • best_lightgbm_seg1.pkl - Segment 1 | LightGBM | Class weights | ROC-AUC: 0.8954
  • best_catboost_seg2.pkl - Segment 2 | CatBoost | No balancing | ROC-AUC: 0.8769
  • best_lightgbm_seg2.pkl - Segment 2 | LightGBM | No balancing | ROC-AUC: 0.8649

СЛЕДУЮЩИЙ ШАГ: Сравнительный анализ и визуализация результатов
